<a href="https://colab.research.google.com/github/gutofranz/Colab_FastAI_Flow/blob/master/Colab_FastAI_Flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## Tirar os '#' e rodar cada bloco separado no terminal pra não desconectar enquanto estiver rodando

#function ConnectButton(){
#    console.log("Working"); 
#    document.querySelector("#connect​").click() 
#}

#setInterval(ConnectButton,60000);

In [9]:
#Importacao via python da biblioteca fastai
! [ -e /content ] && pip install -Uqq fastai  

# funcionou com essa flag comentada, se continuar assim vou excluir
#%matplotlib inline

#Importações do google colab
from google.colab import drive
from google.colab import widgets as colabwidigets
from google.colab import output  as colaboutputs

#Importações Ipywidigets
from ipywidgets import widgets as ipywidgets

#Importações Fastai
from fastai.data.all import *
from fastai.vision.core import *
from fastai.vision    import *
from fastai.vision.all import *

#Importação Pil
from PIL import Image

# Função para a conexão com o GDrive
def drive_connect():
  drive.mount('/content/gdrive', force_remount=True)
  drive_connect.root_dir = "/content/gdrive/My Drive/"
  print('Google Drive conectado com sucesso')

def set_path():
  set_path.path = None

def set_dls():
  set_dls.dls = None 

def set_learner():
  set_learner.learner = None
  set_learner.interp = None


# Função para a criação do DataLoader
def build_data(path, item_tfms_resize, item_tfms_resize_mtd, splitter_percent_validation, batch_tfms_aug_tfms_size):
  #folder = ['Aroeira-vermelha', 'Embauba', 'Jeriva', 'Olandi', 'Pitangueira'] # temporario
  data = get_image_files(path)#, folders=folder)

  # Splitter
  splitter = RandomSplitter(valid_pct=splitter_percent_validation, seed=40)
  splitter(data)

  # Transformações
  item_tfms = [Resize(item_tfms_resize, method=item_tfms_resize_mtd)]
  batch_tfms=[*aug_transforms(size=batch_tfms_aug_tfms_size), Normalize.from_stats(*imagenet_stats)]  

  # Criação do DataBlock
  blocks=(ImageBlock, CategoryBlock)
  get_image_files(path)    
  trees = DataBlock(blocks=blocks,
                    get_items=get_image_files,
                    splitter=splitter,
                    get_y=parent_label,
                    item_tfms=item_tfms,
                    batch_tfms=batch_tfms)
  
  # Criação do DataLoader
  dls = trees.dataloaders(path, bs=10)
  build_data.dls = dls

# Função para a realização do treinamento
def treinamento(model_architecture, epochs):
  dls = set_dls.dls
  defaults.callbacks[1] = Recorder(train_metrics=True)
  defaults.callbacks
  cbs=[ShowGraphCallback,ActivationStats(with_hist=True),SaveModelCallback]
  learn = None 
  #print(model_architecture_value)
  learn = cnn_learner(dls,model_architecture,metrics=(error_rate, accuracy),cbs=cbs)
  #learn.show_training_loop()
  learn.fit_one_cycle(epochs,1e-2)
  treinamento.learner = learn

# Função para executar a Interface Visual
def exec():

  # Criação do painel que vai conter as abas
  tab = colabwidigets.TabBar(['Dados', 'Transfer Learning', 'Av. des. TL', 'Fine Tuning', 'Av. Des. FT', 'Predição', 'Deployment'])

  # Execução da primeira aba: Dados
  with tab.output_to(0):
    
    # Executa a conexão com o GDrive
    drive_connect()
    
    # Criação do grid que vai conter as etapas e entradas/saídas
    grid = colabwidigets.Grid(14, 1)  

    # Atividade 1 - Definir Path
    with grid.output_to(0,0):
      print ('1 - Definir Path')

      display('Dataset/OK') ## temporario

      input_path = ipywidgets.Text(placeholder='Insira o path', value='')
      btn_path = ipywidgets.Button(description='Carregar Path')

      display(input_path, btn_path)

      def on_btn_path_clicked(b):
        # Saída - Atividade 1
        with grid.output_to(1,0):
          grid.clear_cell()
          set_path.path = drive_connect.root_dir + input_path.value
          print('Path:\'' + set_path.path + '\' definido com sucesso!')
      
      btn_path.on_click(on_btn_path_clicked)  

    # Atividade 2 - Tratamento de imagens
    with grid.output_to(2,0):
      print ('2 - Tratamento de imagens')
      
      item_tfms_resize = ipywidgets.IntSlider(min = 8, max = 640, step = 8, value = 456, description = "ITEM_TFMS RESIZE:")
      item_tfms_resize_mtd = ipywidgets.Dropdown(options=['crop', 'pad', 'squish'],value = 'crop',description = 'Metodo item_tfms_resize')

      display(item_tfms_resize, item_tfms_resize_mtd)
    
    # Atividade 3 - Tratamento corretivo de imagen
    with grid.output_to(3,0):
      print ('3 - X - Tratamento corretivo de imagen')
      
    # Atividade 4 - Separação de conjunto de dados de treinamento e validação
    with grid.output_to(4,0):
      print ('4 - Separação de conjunto de dados de treinamento e validação')
      
      splitter_percent_validation = ipywidgets.FloatSlider(min = 0, max = 1, step = 0.01, value = 0.2, description = "% Validação")

      display(splitter_percent_validation)

    # Atividade 5 - Tratamento do dataset: augm_transform
    with grid.output_to(5,0):
      print ('5 - DEV - Tratamento do dataset: augm_transform')
      
      batch_tfms_aug_tfms_size = ipywidgets.IntSlider(min = 128, max = 480, step = 8, value = 224, description = "Batch_TFMS AUG_TFM_SIZE:")

      display(batch_tfms_aug_tfms_size)

    # Atividade 6 - Especificar batch size
    with grid.output_to(6,0):
      print ('6 - X - Especificar batch size')

    # Atividade 7 - Criar dataloader
    with grid.output_to(7,0):
      print ('7 - DEV - Criar dataloader')    

      btn_build_data = ipywidgets.Button(description='Criar Dataloader')

      display(btn_build_data)

      def on_btn_build_data_clicked(b):
        # Saída - Atividade 6
        with grid.output_to(8,0):
          build_data(set_path.path, item_tfms_resize.value, item_tfms_resize_mtd.value, splitter_percent_validation.value, batch_tfms_aug_tfms_size.value)
          set_dls.dls = build_data.dls
          print('Dataloader criado com sucesso!')
          print('Parametros informados:')
          print('Path = ' + set_path.path)
          print('item_tfms_resize = ' + str(item_tfms_resize.value))
          print('item_tfms_resize_mtd = ' + str(item_tfms_resize_mtd.value))
          print('batch_tfms_aug_tfms_size = ' + str(batch_tfms_aug_tfms_size.value))
          print('% Valid. Spl. = ' + str(splitter_percent_validation.value))
 
      btn_build_data.on_click(on_btn_build_data_clicked)  
    
    # Atividade 8 - Visualizar batch
    with grid.output_to(9,0):
      print ('8 - Visualizar batch')

      slider_show_batch = ipywidgets.IntSlider(min = 1, max = 10, step = 1, value = 3, description = "Itens do batch")
      btn_show_batch = ipywidgets.Button(description='Ver batch')

      display(slider_show_batch, btn_show_batch)
    
      def on_btn_show_batch_clicked(b):
        # Saída - Atividade 8
        with grid.output_to(10,0):
          grid.clear_cell()
          y = build_data.dls
          print('Carregando batch, aguarde...')
          y.show_batch(max_n=slider_show_batch.value)
        
      btn_show_batch.on_click(on_btn_show_batch_clicked)  

    # Atividade 9 - Visarlizar infos do dataset
    with grid.output_to(11, 0):
      print ('9 - X - Visarlizar infos do dataset')

    # Atividade 10 - Visualizar as imagens do dataset / datacleaning
    with grid.output_to(12, 0):
      print ('10 - X - Visualizar as imagens do dataset / datacleaning')

    # Atividade 11 - Novas definiccoes
    with grid.output_to(13, 0):
      print ('11 - X - Novas definiccoes')
  
  # Execução da segunda aba: Treinamento do modelo - #1 Transfer Learning
  with tab.output_to(1, select=False):  

    # Criação do grid que vai conter as etapas e entradas/saídas
    grid2 = colabwidigets.Grid(5,1)

    # Atividade 12 - Arquitetura
    with grid2.output_to(0,0):
      print ('12 - Arquitetura')

      model_architecture = ipywidgets.Dropdown(options=[('alexnet', models.alexnet),
                                                      ('densenet121', models.densenet121),
                                                      ('densenet161', models.densenet161),
                                                      ('densenet169', models.densenet169),
                                                      ('densenet201', models.densenet201),
                                                      ('resnet18', models.resnet18),
                                                      ('resnet34', models.resnet34),
                                                      ('resnet50', models.resnet50),
                                                      ('resnet101', models.resnet101),
                                                      ('resnet152', models.resnet152),
                                                      ('squeezenet1_0', models.squeezenet1_0),
                                                      ('squeezenet1_1', models.squeezenet1_1),
                                                      ('vgg16_bn', models.vgg16_bn),
                                                      ('vgg19_bn',models.vgg19_bn)],
                                               description ='Arquitetura:',
                                               value = models.vgg19_bn)
      
      display(model_architecture)

    # Atividade - 13 - Epochs
    with grid2.output_to(1,0):
      print ('13 - DEV - Epochs')

      slider_epochs = ipywidgets.IntSlider(min = 1, max = 50, step = 1, value = 3, description = "Epochs")
    
      display(slider_epochs)

    # Atividade - 14 - Learning rate e momentum
    with grid2.output_to(2,0):
      print ('14 - X - Learning rate e momentum')

    # Atividade - 15- Executar Treinamento
    with grid2.output_to(3,0):
      print ('15 - Executar Treinamento')  

      btn_train_model = ipywidgets.Button(description='Rodar treinamento')    

      display(btn_train_model)

      def on_btn_train_model_clicked(b):
        # Saída - Atividade 15
        with grid2.output_to(4,0):
          grid2.clear_cell()
          print('-- Rodando treinamento -- ')
          print('Arquitetura = ' + str(model_architecture.value))
          print('Epochs = ' + str(slider_epochs.value))
          treinamento(model_architecture.value, slider_epochs.value)   
          set_learner.learner = treinamento.learner
          set_learner.interp = ClassificationInterpretation.from_learner(treinamento.learner)
 
      btn_train_model.on_click(on_btn_train_model_clicked)

  # Execução da terceira aba: Avaliação do desempenho do Transfer Learning
  with tab.output_to(2, select=False):  

    # Criação do grid que vai conter as etapas e entradas/saídas
    grid3 = colabwidigets.Grid(18,1)

    # Atividade - 16 - X - Matriz de confusao
    with grid3.output_to(0,0):
      print('16 - X - Matriz de confusao')

      btn_show_c_matrix = ipywidgets.Button(description='Ver Matriz de confusão')

      display(btn_show_c_matrix)
    
      def on_btn_show_c_matrix(b):
        # Saída - Atividade 8
        with grid3.output_to(1,0):
          grid3.clear_cell()
          y = set_learner.interp
          print('Carregando Matriz de confusão, aguarde...')
          grid3.clear_cell()
          y.plot_confusion_matrix()
        
      btn_show_c_matrix.on_click(on_btn_show_c_matrix)  

    # Atividade - 17 - Acuracia por categoria
    with grid3.output_to(2,0):
      print('17 - X - Acuracia por categoria')

    # Atividade - 18 - top losses
    with grid3.output_to(3,0):
      print('18 - X - top losses')
      slider_top_losses = ipywidgets.IntSlider(min = 1, max = 8, step = 1, value = 2, description = "Principais perdas")
      btn_show_top_losses = ipywidgets.Button(description='Ver principais perdas')      

      display(slider_top_losses, btn_show_top_losses)

      def on_btn_show_top_losses(b):
        #Saída Atividade 19
        with grid3.output_to(4,0):
          y = set_learner.interp
          print('Carregando principais perdas, aguarde...')
          y.plot_top_losses(slider_top_losses.value)  
          print('carregou')

      btn_show_top_losses.on_click(on_btn_show_top_losses)               

    # Atividade - 19 - most confused
    with grid3.output_to(5,0):
      print('19 - X - most confused')  

      slider_most_confused = ipywidgets.IntSlider(min = 1, max = 8, step = 1, value = 1, description = "Top most confused")
      btn_show_most_confused = ipywidgets.Button(description='Ver mais confusos')      

      display(slider_most_confused, btn_show_most_confused)

      def on_btn_show_most_confused(b):
        #Saída Atividade 19
        with grid3.output_to(6,0):
          y = set_learner.interp
          print('Carregando itens mais confusos, aguarde...')
          print(y.most_confused(min_val=slider_most_confused.value))
          print('carregou')

      btn_show_most_confused.on_click(on_btn_show_most_confused) 

    # Atividade - 20 - Heatmaps
    with grid3.output_to(7,0):
      print('20 - X - Heatmaps')  
    
    # Atividade - 21 - Visualizações alternativa  
    with grid3.output_to(8,0):
      print('21 - X - Visualizações alternativa')  
    
    # Atividade - 22 - Data cleaning (de novo?)
    with grid3.output_to(9,0):
      print('22 - X - Data cleaning (de novo?)')  
    
    # Atividade - 
    with grid3.output_to(10,0):
      print('23 - X - Imagens no validation set')  

    # Atividade - X
    with grid3.output_to(11,0):
      print('24 - X - Salvar Modelo')  

      btn_save_model = ipywidgets.Button(description='Salvar modelo')      

      display(btn_save_model)

      def on_btn_save_model(b):
        #Saída Atividade 24
        with grid3.output_to(12,0):
          y = set_learner.learner
          print('Salvando o modelo...')
          y.save('model1')
          print('salvou')

    btn_save_model.on_click(on_btn_save_model) 

  # Execução da quarta aba: Treinamento do modelo - #2 Fine Tuning
  with tab.output_to(3, select=False):  

    # Criação do grid que vai conter as etapas e entradas/saídas
    grid4 = colabwidigets.Grid(5,1)

    # Atividade 12 - Arquitetura
    with grid4.output_to(0,0):

      btn_exec2 = ipywidgets.Button(description='Carregar Modelo')
      display(btn_exec2)

      def on_btn_exec2(b):
        with grid4.output_to(1,0):
          print('Set da variavel')
          y = set_learner.learner
          print('Modelo carregado')
          y.load('model1')
          print('lr_find()')
          y.lr_find()
          print('plot_lr_find()')
          y.recorder.plot_lr_find()
          print('salvando modelo')
          y.save('model1')
          print('unfreeze')
          y.unfreeze()
          print('max_lr=slice...')
          max_lr=slice(1e-4, 1e-3)
          print('fit_one_cycle(3,max_lr)')
          y.fit_one_cycle(3,max_lr)
          print('Salvando o modelo')
          y.save('model1')

      btn_exec2.on_click(on_btn_exec2)
      








  # Execução da quinta aba: Avaliação do desempenho do Fine Tuning
  with tab.output_to(4, select=False):  

    # Criação do grid que vai conter as etapas e entradas/saídas
    grid5 = colabwidigets.Grid(18,1)

    # Atividade - 16 - X - Matriz de confusao
    with grid5.output_to(0,0):
      print('16 - X - Matriz de confusao')

      btn_show_c_matrix_ft = ipywidgets.Button(description='Ver Matriz de confusão')

      display(btn_show_c_matrix_ft)
    
      def on_btn_show_c_matrix_ft(b):
        # Saída - Atividade 8
        with grid5.output_to(1,0):
          grid5.clear_cell()
          y = set_learner.interp
          print('Carregando Matriz de confusão, aguarde...')
          grid5.clear_cell()
          y.plot_confusion_matrix()
        
      btn_show_c_matrix_ft.on_click(on_btn_show_c_matrix_ft)  

    # Atividade - 17 - Acuracia por categoria
    with grid5.output_to(2,0):
      print('17 - X - Acuracia por categoria')

    # Atividade - 18 - top losses
    with grid5.output_to(3,0):
      print('18 - X - top losses')
      slider_top_losses_ft = ipywidgets.IntSlider(min = 1, max = 8, step = 1, value = 2, description = "Principais perdas")
      btn_show_top_losses_ft = ipywidgets.Button(description='Ver principais perdas')      

      display(slider_top_losses_ft, btn_show_top_losses_ft)

      def on_btn_show_top_losses_ft(b):
        #Saída Atividade 19
        with grid5.output_to(4,0):
          y = set_learner.interp
          print('Carregando principais perdas, aguarde...')
          y.plot_top_losses(slider_top_losses_ft.value)  
          print('carregou')

      btn_show_top_losses_ft.on_click(on_btn_show_top_losses_ft)               

    # Atividade - 19 - most confused
    with grid5.output_to(5,0):
      print('19 - X - most confused')  

      slider_most_confused_ft = ipywidgets.IntSlider(min = 1, max = 8, step = 1, value = 1, description = "Top most confused")
      btn_show_most_confused_ft = ipywidgets.Button(description='Ver mais confusos')      

      display(slider_most_confused_ft, btn_show_most_confused_ft)

      def on_btn_show_most_confused_ft(b):
        #Saída Atividade 19
        with grid5.output_to(6,0):
          y = set_learner.interp
          print('Carregando itens mais confusos, aguarde...')
          print(y.most_confused(min_val=slider_most_confused_ft.value))
          print('carregou')

      

    # Atividade - 20 - Heatmaps
    with grid5.output_to(7,0):
      print('20 - X - Heatmaps')  
    
    # Atividade - 21 - Visualizações alternativa  
    with grid5.output_to(8,0):
      print('21 - X - Visualizações alternativa')  
    
    # Atividade - 22 - Data cleaning (de novo?)
    with grid5.output_to(9,0):
      print('22 - X - Data cleaning (de novo?)')  
    
    # Atividade - 
    with grid5.output_to(10,0):
      print('23 - X - Imagens no validation set')  

    # Atividade - X
    with grid5.output_to(11,0):
      print('24 - X - Salvar Modelo')  

      btn_save_model_ft = ipywidgets.Button(description='Salvar modelo')      

      display(btn_save_model_ft)

      def on_btn_save_model_ft(b):
        #Saída Atividade 24
        with grid5.output_to(12,0):
          y = set_learner.learner
          print('Carregando o modelo...')
          y.load('model1')
          print('Carregado')

      btn_save_model_ft.on_click(on_btn_save_model_ft) 




  # Execução da sexta aba: Predição
  with tab.output_to(5, select=False):  

    # Criação do grid que vai conter as etapas e entradas/saídas
    grid6 = colabwidigets.Grid(18,1)

    # Atividade - 
    with grid6.output_to(1,0):
      print('23 - X - Carregar Modelo')  

      btn_load_model = ipywidgets.Button(description='Carregar Modelo')
      display(btn_load_model)

      def on_btn_load_model(b):
        with grid6.output_to(2,0):
          print('Set da variavel')
          y = set_learner.learner
          print('Modelo carregado')
          y.load('model1')

      btn_load_model.on_click(on_btn_load_model) 

    # Atividade - 
    with grid6.output_to(3,0):

      btn_file_upload = ipywidgets.FileUpload()
      btn_pred = ipywidgets.Button(description='Predição')
      display(btn_file_upload, btn_pred)

      def on_btn_pred(b):
        with grid6.output_to(4,0):
          print('Set da variavel')
          y = set_learner.learner
          print('Modelo carregadoo')
          y.load('model1')
          print('Criando a imagem no Pil')
          img = PILImage.create(btn_file_upload.data[0])
          print('Fazendo predict da imagem')
          flag1,_,probs = y.predict(img)
          print(flag1)
          print(probs[1].item())
        

      btn_pred.on_click(on_btn_pred)



  # Execução da sétima aba: Deployment
  with tab.output_to(6, select=False):  

    # Criação do grid que vai conter as etapas e entradas/saídas
    grid7 = colabwidigets.Grid(18,1)

        # Atividade - 
    with grid7.output_to(1,0):

      btn_export_1 = ipywidgets.Button(description = 'Exportar modelo')
      display(btn_export_1)

      def on_btn_export_1(b):
        with grid7.output_to(4,0):
          print('Set da variavel')
          y = set_learner.learner
          print('exportando')
          y.export()
          print('foi')
          
      btn_export_1.on_click(on_btn_export_1)


print('foi')    

foi


In [ ]:
exec()